## Step 2 - Data Augmentation

In [1]:
# Running Tensorboard

%load_ext tensorboard
%tensorboard --logdir=./mylogs

Reusing TensorBoard on port 6008 (pid 23112), started 23:18:06 ago. (Use '!kill 23112' to kill it.)

In [35]:
# Creating image generators

from keras.preprocessing.image import ImageDataGenerator

image_shape = (240, 240, 3) # I want the images to be square and not to big to speed up training.
test_split_size = 0.15
batch_size = 128
rescale_factor = 1/255

# Define the parameters for data augmentation
train_datagen = ImageDataGenerator(
    rescale=rescale_factor,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)


# Create the training dataset
train_dataset = train_datagen.flow_from_directory(
    'data/flowers_cleaned_split/train',
    target_size=image_shape[:2],
    batch_size=batch_size,
    class_mode='categorical'
)

val_datagen = ImageDataGenerator(rescale=1./255)

# Create the val dataset
val_dataset = val_datagen.flow_from_directory(
    'data/flowers_cleaned_split/val/',
    target_size=image_shape[:2],
    batch_size=batch_size,
    class_mode='categorical'
)

test_datagen = ImageDataGenerator(rescale=1./255)

# Create the test dataset
test_dataset = test_datagen.flow_from_directory(
    'data/flowers_cleaned_split/test/',
    target_size=image_shape[:2],
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)


print(f'Number Of Images In Training Dataset : {str(train_dataset.samples)}')
print(f'Number Of Images In Validation Dataset : {str(val_dataset.samples)}')
print(f'Number Of Images In Validation Dataset : {str(test_dataset.samples)}')
print(f'Total number of images: {train_dataset.samples+val_dataset.samples+test_dataset.samples}')

Found 2915 images belonging to 5 classes.
Found 624 images belonging to 5 classes.
Found 628 images belonging to 5 classes.
Number Of Images In Training Dataset : 2915
Number Of Images In Validation Dataset : 624
Number Of Images In Validation Dataset : 628
Total number of images: 4167


## Step 3 - Creating the model

In [36]:
# Defining number of steps for training & validation of model
steps_per_epoch = train_dataset.samples // batch_size
validation_steps = val_dataset.samples // batch_size

print('Steps Per Epoch : Training -> ' + str(steps_per_epoch))
print('Steps : Validation -> ' + str(validation_steps))

Steps Per Epoch : Training -> 22
Steps : Validation -> 4


In [37]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Dropout, Activation, BatchNormalization
from keras import regularizers
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, TensorBoard
from pathlib import Path
from time import strftime

In [14]:
# Callbacks

# Defining tensorboard callback for logging and visualization
def get_run_logdir(custom_text="", root_logdir="my_logs"):
    formatted_time = strftime("run_%Y_%m_%d_%H_%M_%S")
    if custom_text:
        custom_text = "_" + custom_text
    return Path(root_logdir) / (formatted_time + custom_text)


run_logdir = get_run_logdir("added_1_more_conv")

tensorboard_cb = TensorBoard(run_logdir, profile_batch=(100, 200))

# Defining earlystopping callback to stop training if model isnt getting better
early_stop_cb = EarlyStopping(monitor="val_loss", patience=10)
run_logdir

WindowsPath('my_logs/run_2023_12_13_22_49_57_added_1_more_conv')

In [15]:
# create model
model = Sequential()

# add layers to model
model.add(
    Conv2D(
        filters=32,
        padding="same",
        kernel_size=(3, 3),
        input_shape=image_shape,
        activation="relu",
    )
)

model.add(
    Conv2D(
        filters=32,
        padding="same",
        kernel_size=(3, 3),
    )
)

model.add(MaxPool2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.2))

model.add(
    Conv2D(
        filters=64,
        padding="same",
        kernel_size=(3, 3),
    )
)

model.add(Conv2D(filters=64, padding="same", kernel_size=(3, 3), activation="relu"))

model.add(MaxPool2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.2))

model.add(
    Conv2D(
        filters=128,
        padding="same",
        kernel_size=(3, 3),
    )
)

model.add(Conv2D(filters=128, padding="same", kernel_size=(3, 3), activation="relu"))
model.add(Conv2D(filters=128, padding="same", kernel_size=(3, 3), activation="relu"))

model.add(MaxPool2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.2))

model.add(Flatten())
model.add(Dense(512, activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(5, activation="softmax"))

# Setting compilation parameters
learning_rate = 0.001
optimizer = Adam(learning_rate=learning_rate)
model.compile(
    loss="categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"]
)

In [16]:
# Checking model summary
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 240, 240, 32)      896       
                                                                 
 conv2d_1 (Conv2D)           (None, 240, 240, 32)      9248      
                                                                 
 max_pooling2d (MaxPooling2  (None, 120, 120, 32)      0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 120, 120, 32)      0         
                                                                 
 conv2d_2 (Conv2D)           (None, 120, 120, 64)      18496     
                                                                 
 conv2d_3 (Conv2D)           (None, 120, 120, 64)      36928     
                                                        

In [ ]:
# Launching Tensorboard
%load_ext tensorboard
%tensorboard --logdir=./mylogs

In [22]:
early_stop_cb = EarlyStopping(monitor="val_loss", patience=10)

## Step 4 - Training the model

In [23]:
# Training model
history = model.fit(train_dataset,epochs=10,steps_per_epoch = steps_per_epoch,
                    validation_data = val_dataset, validation_steps=validation_steps,
                    verbose=True, callbacks=[early_stop_cb, tensorboard_cb])

Epoch 1/10
22/22 [==============================] - 262s 11s/step - loss: 0.6607 - accuracy: 0.7463 - val_loss: 0.6692 - val_accuracy: 0.7617
Epoch 2/10
22/22 [==============================] - 236s 10s/step - loss: 0.6489 - accuracy: 0.7503 - val_loss: 0.6467 - val_accuracy: 0.7812
Epoch 3/10
22/22 [==============================] - 268s 12s/step - loss: 0.6536 - accuracy: 0.7499 - val_loss: 0.6632 - val_accuracy: 0.7754
Epoch 4/10
22/22 [==============================] - 220s 10s/step - loss: 0.6548 - accuracy: 0.7510 - val_loss: 0.6750 - val_accuracy: 0.7715
Epoch 5/10
22/22 [==============================] - 223s 10s/step - loss: 0.6436 - accuracy: 0.7610 - val_loss: 0.6518 - val_accuracy: 0.7520
Epoch 6/10
22/22 [==============================] - 231s 10s/step - loss: 0.6489 - accuracy: 0.7607 - val_loss: 0.6526 - val_accuracy: 0.7715
Epoch 7/10
22/22 [==============================] - 304s 14s/step - loss: 0.6276 - accuracy: 0.7614 - val_loss: 0.6679 - val_accuracy: 0.7676
Epoch 

In [25]:
model.evaluate(test_dataset)

5/5 [==============================] - 12s 2s/step - loss: 0.6303 - accuracy: 0.7596


[0.6303080320358276, 0.7595541477203369]

## Svar på frågor:
- Motivera din modellarkitektur och val av relevanta hyperparametrar.  
    Genom experiment med olika data augmentation, regularisering, lagerstorlek och annat så märkte jag att det som gjorde störst positiv skillnad vadrantal conv-lager. 
- Vilka aktiveringsfunktioner har du använt? Varför?   
    ReLu - pga standard, ej experimenterat mycket med detta.
- Vilken loss funktion har du använt? Varför?  
    Adam - pga standard, ej experimenterat mycket här.
- Har du använt någon databehandling? Varför?   
    Ja, jag har provat ganska mycket augmentation eftersom datasetet är relativt litet. Rotation, spegelvändning, shear och zoomning känns allihopa som att det inte förstör bilden. Möjligen blir bilder där människor ingår lite konstiga vid rotation, men det borde vara försumbart.
- Har du använt någon regulariseringsteknik? Motivera.   
    Jag har provat L1 och L2, batch normalization samt Dropout. Det som fungerade bäst var Dropout och det var det enda jag behöll till sist.
- Hur har modellen utvärderats?  
    Jag har använt Tensorboard och kört modellerna cirka 10 epoker åt gången, jag körde för det mesta early stopping på "val_loss" med patience 5 för att inte fastna för länge på en dålig modell.
- Är prestandan bra? Varför/ varför inte?   
    Jag kom till slut upp till cirka 0.75 accuracy på testdatan vilket jag är nöjd med.
- Vad hade du kunnat göra för att förbättra den ytterligare?  
    Antagligen göra ännu fler convlager eller denselager i slutet. Då skulle det dock börja ta lite för lång tid att träna.